In [1]:
# Files:
# natural_units.py
# quench_worker.py
# simulation_module.py
# utility_functions.py
# laser.py
import os
import numpy as np
from simulation_module import SimulationConfig, SimulationState, SimulationRunner, SimulationIO, SimulationVisualizer, AnimationMaker
from quench_worker import run_single_quench
from laser import Laser

In [2]:
laser650 = laser650 = Laser(650, [1,0], saturation = 10, detuning = -2 * np.pi * 20 / 2, Gamma = 2 * np.pi * 20)

In [3]:
config = SimulationConfig(
    N = 6,
    w = 1.0,
    g = 1.0,
    m = 137.327,  # mass in atomic units
    T = 0.,     # temperature in mK
    dt = 1e-3,
    num_steps = int(1e6),
    damping = False,
    damping_parameter = 1.0,
    langevin_temperature = False,
    lasers=[laser650]  # or a list of Laser instances
)

In [4]:
state = SimulationState(config)
state.positions = SimulationIO.load_positions("6_1_positions_1.json")
state.initial_positions = state.positions.copy()
state.initialized = True

In [5]:
from dataclasses import asdict
from concurrent.futures import ProcessPoolExecutor
from quench_worker import run_single_quench

def run_quench_series_parallel(config, loadfile, output_dir,
                               g_start, g_end, g_step, num_workers=4):
    os.makedirs(output_dir, exist_ok=True)
    gammas = np.linspace(g_start, g_end, g_step)
    base_config_dict = asdict(config)
    print("Beginning to parralel quench")
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = []
        for gamma in gammas:
            futures.append(executor.submit(run_single_quench, gamma, base_config_dict, loadfile, output_dir))
        for f in futures:
            f.result()

    print("All quench simulations completed.")

In [6]:
quench_trajectory_folder = "3quench_data"
animation_folder = "3animation"
temperature_folder = "3temperature"

In [11]:
run_quench_series_parallel(
    config = config,
    loadfile = "6_1_positions_1.json",
    output_dir = quench_trajectory_folder,
    g_start = 0.86,
    g_end = 0.82,
    g_step = 50,
    num_workers = 7
)

Beginning to parralel quench
Skipping gamma = 0.85755102, file exists.
Skipping gamma = 0.83795918, file exists.
Skipping gamma = 0.83714286, file exists.
Skipping gamma = 0.83551020, file exists.
Skipping gamma = 0.83387755, file exists.
Skipping gamma = 0.83306122, file exists.
Skipping gamma = 0.83224490, file exists.
Skipping gamma = 0.83061224, file exists.
Skipping gamma = 0.82897959, file exists.
Skipping gamma = 0.82816327, file exists.
Skipping gamma = 0.82734694, file exists.
Skipping gamma = 0.82571429, file exists.
Skipping gamma = 0.82408163, file exists.
Skipping gamma = 0.82244898, file exists.
Skipping gamma = 0.82081633, file exists.
Skipping gamma = 0.86000000, file exists.
Running gamma = 0.84857143
Running with isotropy (γ) = 0.8486 for 1000.0 μs
Skipping gamma = 0.85591837, file exists.
Skipping gamma = 0.84775510, file exists.
Skipping gamma = 0.84693878, file exists.
Skipping gamma = 0.84612245, file exists.
Skipping gamma = 0.84530612, file exists.
Skipping gamm

In [ ]:
# Step 1: Convert trajectories to images
AnimationMaker.generate_density_map_images_from_quench_folder(quench_trajectory_folder, animation_folder, config)

# Step 2: Turn into animation
AnimationMaker.make_gif_or_mp4_from_images(animation_folder, "-quench_animation.mp4", fps=6)

Processing trajectory files
Processing 3quench_data/6_0.86000000000000_traj_1000000_steps.json
Processing 3quench_data/6_0.85918367346939_traj_1000000_steps.json
Processing 3quench_data/6_0.85836734693878_traj_1000000_steps.json
Processing 3quench_data/6_0.85755102040816_traj_1000000_steps.json
Processing 3quench_data/6_0.85673469387755_traj_1000000_steps.json
Processing 3quench_data/6_0.85591836734694_traj_1000000_steps.json
Processing 3quench_data/6_0.85510204081633_traj_1000000_steps.json
Processing 3quench_data/6_0.85428571428571_traj_1000000_steps.json


In [12]:
AnimationMaker.generate_temperature_plots_from_quench_folder(
    traj_folder = quench_trajectory_folder,
    output_dir = temperature_folder,
    base_config = config,
    grainyness= 100)

✅ All temperature plots generated.


In [ ]:
# # Step 1: Convert trajectories to images
# AnimationMaker.generate_density_map_images_from_quench_folder(quench_trajectory_folder, animation_folder, config)

# Step 2: Turn into animation
AnimationMaker.make_gif_or_mp4_from_images(temperature_folder, "-temperature_animation.mp4", fps=6)